In [1]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011
     |████████████████████████████████| 92kB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 829kB 8.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Packages installed.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


Scrape the postal codes of Canada using Beautiful Soap

In [3]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(wiki_url) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

Data Collection

In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                

In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


Using the Geocoder API

In [6]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [7]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M2A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
2,M3A\n,North York\n,Parkwoods\n,43.75188,-79.33036
3,M4A\n,North York\n,Victoria Village\n,43.73042,-79.31282
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65514,-79.36265
...,...,...,...,...,...
174,M4Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
175,M5Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
176,M6Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
177,M7Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544


In [8]:
neighbors.shape

(179, 5)